In [ ]:
#GEE Authentication and Initialization

In [2]:
import ee
import geemap

In [3]:
ee.Authenticate()
ee.Initialize(project='fleet-hawk-463015-h5')

Enter verification code:  4/1AVMBsJhGUcN37NGwtGtBLB9frBz2YDhqJPUA3utDdDy82hKzf4fmgfSZBcE



Successfully saved authorization token.


In [ ]:
#feature Collection Functions:

In [ ]:
#ndwi from a point+Area Calculation

In [4]:
#------------Final Detect_lake_from_point------------#ddd
import ee
import pandas as pd


def detect_lake_from_point(point, year, search_radius=3000, ndwi_thresh=0.25):
    aoi = point.buffer(search_radius)

    def mask_landsat_clouds(img):
        qa = img.select('QA_PIXEL')
        cloud        = 1 << 3
        cloud_shadow = 1 << 4
        snow         = 1 << 5
        mask = (qa.bitwiseAnd(cloud).eq(0)
                  .And(qa.bitwiseAnd(cloud_shadow).eq(0))
                  .And(qa.bitwiseAnd(snow).eq(0)))
        return img.updateMask(mask)

    def scale_sr(img):
        optical = img.select('SR_B[0-9]*').multiply(0.0000275).add(-0.2)
        return img.addBands(optical, overwrite=True)

    # --- NDWI depending on collection ---
    def add_ndwi(img, sensor):
        if sensor in ["LC08", "LC09"]:   # Landsat 8/9
            green, nir = 'SR_B3', 'SR_B5'
        else:                            # Landsat 5/7
            green, nir = 'SR_B2', 'SR_B4'
        ndwi = img.normalizedDifference([green, nir]).rename('NDWI')
        return img.addBands(ndwi)


    # --- Choose the correct Landsat collection ---
    def get_collection_id(y):
        if y <= 2011:
            return "LANDSAT/LT05/C02/T1_L2"   # Landsat 5
        elif y == 2012:
            return "LANDSAT/LE07/C02/T1_L2"   # Landsat 7
        else:
            return "LANDSAT/LC08/C02/T1_L2"   # Landsat 8/9

    def get_fall_collection(y):
        coll_id = get_collection_id(y)
        sensor = coll_id.split("/")[1][:4]  # e.g. LC08, LT05, LE07
        return (ee.ImageCollection(coll_id)
            .filterDate(f"{y}-08-01", f"{y}-12-31")
            .filterBounds(aoi)
            .map(mask_landsat_clouds)
            .map(scale_sr)
            .map(lambda img: add_ndwi(img, sensor)))

    # --- Combine 3 years ---
    collection = (get_fall_collection(year-1)
                  .merge(get_fall_collection(year))
                  .merge(get_fall_collection(year+1)))

    if collection.size().getInfo() == 0:
        return {'lake_fc': ee.FeatureCollection([]), 'area_ha': None}

    # --- Median composite ---
    ndwi_median = collection.select('NDWI').median().clip(aoi)
    water_mask = ndwi_median.gt(ndwi_thresh)

    lake_fc_all = water_mask.selfMask().reduceToVectors(
        geometry=aoi,
        scale=30,
        geometryType='polygon',
        reducer=ee.Reducer.countEvery(),
        maxPixels=1e13
    )

    def add_area(f):
        return f.set('area_ha', f.geometry().area(maxError=1).divide(1e4))
    lake_fc_all = lake_fc_all.map(add_area)

    largest = ee.Feature(lake_fc_all.sort('area_ha', False).first())

    return {
        'lake_fc': ee.Algorithms.If(
            largest,
            ee.FeatureCollection([largest]),
            ee.FeatureCollection([])
        ),
        'area_ha': ee.Algorithms.If(largest, largest.get('area_ha'), None),
        'ndwi_median': ndwi_median
    }


In [ ]:
#Lake Expansion 

In [5]:
def expansion_rate(point, ref_year, search_radius=3000, ndwi_thresh=0.3):
    """
    Calculate 10-year expansion rate of a non-GLOF lake (ha/year).

    - ref_year: e.g., 2020
    - recent window: (ref_year-1, ref_year, ref_year+1)
    - past window:   (ref_year-11, ref_year-10, ref_year-9)
    - search_radius default set to 3000 m
    """
    
    # --- Recent ---
    recent = detect_lake_from_point(point, ref_year,
                                    search_radius=search_radius,
                                    ndwi_thresh=ndwi_thresh)
    area_recent = recent["area_ha"].getInfo() if recent["area_ha"] else None
    
    # --- Past (10 years earlier) ---
    past_year = ref_year - 10
    past = detect_lake_from_point(point, past_year,
                                  search_radius=search_radius,
                                  ndwi_thresh=ndwi_thresh)
    area_past = past["area_ha"].getInfo() if past["area_ha"] else None
    
    # --- Expansion rate ---
    if area_recent is None or area_past is None:
        print(f"[{ref_year}] Could not compute area(s). "
              f"Recent={area_recent}, Past={area_past}")
        return None
    
    Expansion_rate = (area_recent - area_past) / 10.0   # ha per year
    
    # --- Print for checking ---
    print(f"Ref year {ref_year}: Past={area_past:.2f} ha | "
          f"Recent={area_recent:.2f} ha | "
          f"Expansion={rate:.2f} ha/yr")
    
    return Expansion_rate, area_recent, area_past

In [ ]:
#gets the nearest glacier polygons to lake "polygon" and if not touch just gets the closest glacier's info:

In [14]:
def get_nearest_glacier(lake_geom, buffer_km=50):
    """
    Given a lake polygon, finds nearest or touching glacier(s).
    Returns consistent features for ML:
      - glacier_contact (bool)
      - nearest_glacier_dist_m
      - lake_elev_m
      - glacier_elev_m (from chosen glacier)
      - slope_glac_to_lake (rise/run, slope=0 if touching)
      - glacier_area_km2 (sum of all touching glaciers, or area of nearest one)
      - glacier_ids (list of GLIMS IDs for touching glaciers, or [nearest_id])
    """
    glaciers = ee.FeatureCollection("GLIMS/current")
    dem = ee.Image("USGS/SRTMGL1_003")

    # Filter glaciers within buffer
    nearby = glaciers.filterBounds(lake_geom.buffer(buffer_km * 1000))

    # Glaciers that touch/intersect the lake
    touching = nearby.filterBounds(lake_geom)

    # --- Case 1: touching glaciers ---
    if touching.size().getInfo() > 0:
        # Add area for each
        def add_area(f):
            return f.set("glacier_area_km2", f.geometry().area(maxError=30).divide(1e6))
        touching = touching.map(add_area)

        # Glacier with largest area (used for slope reference)
        largest_touching = ee.Feature(touching.sort("glacier_area_km2", False).first())

        # Collect all glacier IDs
        glacier_ids = touching.aggregate_array("glac_id").getInfo()

        # Metrics
        combined_area = touching.aggregate_sum("glacier_area_km2").getInfo()
        lake_elev = dem.sample(lake_geom.centroid(maxError=30), 30).first().get("elevation").getInfo()
        glac_elev = dem.sample(largest_touching.geometry().centroid(maxError=30), 30).first().get("elevation").getInfo()

        return {
            "glacier_contact": True,
            "nearest_glacier_dist_m": 0,   # touching → distance = 0
            "lake_elev_m": lake_elev,
            "glacier_elev_m": glac_elev,
            "slope_glac_to_lake": 0,      # standardized → slope=0 when touching
            "glacier_area_km2": combined_area,   # sum of all touching glaciers
            "glacier_ids": glacier_ids    # all touching glaciers
        }

    # --- Case 2: no touching glaciers ---
    else:
        # Compute distances
        with_dist = nearby.map(lambda g: g.set("dist", g.geometry().distance(lake_geom, maxError=30)))
        nearest = ee.Feature(with_dist.sort("dist").first())

        # Metrics
        distance = nearest.get("dist").getInfo()
        area_km2 = nearest.geometry().area(maxError=30).divide(1e6).getInfo()
        lake_elev = dem.sample(lake_geom.centroid(maxError=30), 30).first().get("elevation").getInfo()
        glac_elev = dem.sample(nearest.geometry().centroid(maxError=30), 30).first().get("elevation").getInfo()

        slope = (glac_elev - lake_elev) / distance if (lake_elev and glac_elev and distance > 0) else None

        return {
            "glacier_contact": False,
            "nearest_glacier_dist_m": distance,
            "lake_elev_m": lake_elev,
            "glacier_elev_m": glac_elev,
            "slope_glac_to_lake": slope,
            "glacier_area_km2": area_km2,
            "glacier_ids": [nearest.get("glac_id").getInfo()]  # single nearest glacier
        }




In [15]:
#visualize glacier features:

In [16]:
def visualize_glacierE(lake_geom, glacier_info, zoom=11):
    """
    Visualize the lake polygon, glaciers (touching or nearest), and connection line.
    Compatible with the new get_nearest_glacier() output.
    """
    # Base map
    center = lake_geom.centroid(maxError=30).coordinates().getInfo()[::-1]  # [lat, lon]
    m = geemap.Map(center=center, zoom=zoom)
    m.add_basemap("SATELLITE")

    # DEM overlay (SRTM 30m)
    dem = ee.Image("USGS/SRTMGL1_003")
    dem_vis = {
        "min": 4000,
        "max": 6000,
        "palette": ["blue", "green", "yellow", "orange", "red", "white"]
    }
    m.addLayer(dem, dem_vis, "DEM Elevation")

    # Lake polygon
    m.addLayer(lake_geom, {"color": "cyan"}, "Lake Polygon")

    if glacier_info["touching"]:
        # --- Case 1: Touching glaciers ---
        largest_overlap = ee.Feature(glacier_info["largest_overlap_glacier"])
        m.addLayer(largest_overlap.geometry(), {"color": "purple"}, "Largest Overlap Glacier")

    else:
        # --- Case 2: Nearest glacier ---
        nearest = ee.Feature(glacier_info["nearest_glacier"])
        nearest_point = nearest.geometry().centroid(maxError=30)
        lake_centroid = lake_geom.centroid(maxError=30)

        # Buffers
        m.addLayer(lake_centroid.buffer(200), {"color": "black"}, "Lake Centroid")
        m.addLayer(nearest_point.buffer(100), {"color": "red"}, "Nearest Glacier Point")

        # Glacier polygon
        m.addLayer(nearest.geometry(), {"color": "purple"}, "Nearest Glacier")

        # Connection line
        line = ee.Geometry.LineString([
            lake_centroid.coordinates(),
            nearest_point.coordinates()
        ])
        m.addLayer(line, {"color": "yellow", "width": 2}, "Connection Line")

    return m




In [17]:
# --- Test one lake ---
lat, lon = 27.864,87.837   # example coords
point = ee.Geometry.Point([lon, lat])

# 1. Detect lake polygon
lake_result = detect_lake_from_point(point, 2016)
lake_fc = ee.FeatureCollection(lake_result["lake_fc"])
print("Lake polygons detected:", lake_fc.size().getInfo())
# Only print the summary keys we care about
summary_keys = [
    "touching",
    "distance_m",
    "lake_elev_m",
    "glacier_elev_m",
    "slope_glac_to_lake",
    "glacier_area_km2"
]

for k in summary_keys:
    print(f"{k:20}: {glacier_info.get(k)}")

Lake polygons detected: 1


NameError: name 'glacier_info' is not defined

In [18]:
m = visualize_glacierE(lake_geom, glacier_info, zoom=11)
m


NameError: name 'lake_geom' is not defined